Traitement de données massives avec Pyspark
========
DataFrames
--------
- - - -

La structure RDD n'est pas optimisée pour effectuer des tâches par colonne ou du Machine Learning. La structure DataFrame a été créé pour répondre à ce besoin. Elle utilise de façon sous-jacente les bases d'un RDD mais a été structurée en colonnes autant qu'en lignes dans une structure SQL et une forme inspirée des DataFrame du module __pandas__.

La structure DataFrame possède deux grands avantages. Tout d'abord cette structure est similaire au DataFrame __pandas__ et est donc facile à prendre en main. Elle est également performante : un DataFrame en __PySpark__ est aussi rapide qu'un DataFrame en __Scala__ et est la structure distribuée la plus optimisée en Machine Learning. Grâce à la structure DataFrame, nous pouvons donc faire des calculs performants à travers un langage familier, en évitant le coût d'entrée d'apprentissage d'un nouveau langage fonctionnel : __Scala__.

Dans cet exercice, vous apprendrez à manipuler un DataFrame __PySpark__ pour explorer les données.

## 1. Spark SQL

__Spark SQL__ est un module de Spark qui permet de travailler sur des données structurées. C'est donc au sein de ce module qu'a été développé le DataFrame de Spark et __Spark SQL__ vient se rajouter à Spark pour y apporter de nombreux éléments de structure. En particulier, il introduit la notion de <code>SparkSession</code>, originellement le point d'entrée de __Spark SQL__, mais qui est devenu le point d'entrée unifié de Spark.

* __(a)__ Exécuter la cellule ci-dessous pour construire la <code>SparkSession</code> de l'exercice

In [1]:
# Import de Spark Session et SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Définition d'un SparkContext
SparkContext.getOrCreate() 

# Définition d'une SparkSession
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Introduction au DataFrame") \
    .getOrCreate()

spark

Deux options ont été mises en place dans la construction de la <code>SparkSession</code> :
* La méthode <code>appName</code> qui permet de donner à la <code>SparkSession</code> un nom pour connaître l'environnement de travail.
* La méthode <code>getOrCreate</code> qui permet d'utiliser la session en cours si une session est déjà ouverte.

En effet, cette dernière est importante car il n'est pas possible d'ouvrir deux sessions sur une seule machine.<br>
Le nom sert surtout d'indicateur pour connaître la session en cours : si une session existe déjà, elle ne sera pas renommée et affichera l'ancien nom.<br>
<code>SparkSession</code> est une couche supérieure à <code>SparkContext</code>. Ce dernier est avantageux car il permet de travailler avec des RDD.

* __(b)__ Exécuter la cellule ci-dessous pour créer un raccourci vers le <code>SparkContext</code> déjà créé.

In [2]:
# Création d'un raccourci vers le SparkContext déjà créé
sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

<div class="alert alert-success">
Spark SQL possède une documentation https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html en une seule page (idéale pour faire des recherches automatiques à l'aide de CTRL+F) assez riche et qui propose des exemples en plus des explications. Contrairement à ce que vous pouvez trouver sur internet, cette documentation est le seul document régulièrement mis à jour avec la dernière version de Spark.
</div>

## 2. Créer un DataFrame Spark

### 2.1. Créer Un DataFrame à partir d'un RDD

La structure DataFrame a été implémentée par __Spark SQL__, le module __Spark__ pour le traitement de données structurées. Dans sa forme, c'est tout simplement un RDD dont chaque ligne est un <code>Row</code>. Les <code>Rows</code> permettent de donner des noms à chacune des colonnes pour insérer une structure supplémentaire à la donnée.

Par exemple, pour un RDD nommé rdd contenant deux éléments sur chaque ligne, le nom d'un individu et son âge, il est possible d'appliquer sur chaque ligne le constructeur de la classe Row en donnant un nom à chaque variable.

La création d'un DataFrame à partir d'un rdd se fait en appliquant la méthode createDataFrame de SparkContext.

__Exemple : Créer un DataFrame à partir d'un RDD__
```python
rdd_row = rdd.map(lambda line: Row(name = line[0], age = line[1]))
df = spark.createDataFrame(rdd_row)
```

* __(a)__ Importer la classe <code>Row</code> à partir du module __pyspark.sql__.
* __(b)__ Importer la base de données <code>2008_raw.csv</code>.
* __(c)__ Créer un __rdd__ à partir de cette base de données.
* __(d)__ Créer un __rdd_row__ à l'aide de la méthode <code>map</code> en appliquant sur chaque ligne la structure Row avec les variables explicatives annee, mois, jours et flightNum.
* __(e)__ Créer un DataFrame __df__ à partir de __rdd_row__.

In [3]:
# Import de Row du package pyspark.sql
from pyspark.sql import Row

# Chargement du fichier '2008_raw.csv'
rdd = sc.textFile('data/2008_raw.csv').map(lambda line: line.split(","))

# Création d'un nouveau RDD en sélectionnant les variables explicatives
rdd_row = rdd.map(lambda line: Row(annee = line[0],
                                   mois = line[1],
                                   jours = line[2],
                                   flightNum = line[5]))

# Créer d'un data frame à partir d'un rdd
df = spark.createDataFrame(rdd_row)

Il est important de savoir afficher quelques lignes d'un DataFrame pour vérifier que le tableau de données est bien construit. De plus, il est pertinent d'utiliser la méthode <code>show(n)</code> qui affiche de façon claire les n premières lignes du DataFrame.

<div class="alert alert-success">
Il est possible d'exécuter la méthode take qui affiche les n premières lignes en revenant sous la forme d'un RDD de Rows peu lisible.
</div>

* __(f)__ Afficher les 5 premières lignes de df

In [4]:
# Affichage des 5 premières lignes
df.show(5)

+-----+----+-----+---------+
|annee|mois|jours|flightNum|
+-----+----+-----+---------+
| 2008|   1|    1|     2052|
| 2008|   1|    1|      715|
| 2008|   1|    1|     1846|
| 2008|   1|    1|     2300|
| 2008|   1|    1|     1221|
+-----+----+-----+---------+
only showing top 5 rows



<div class="alert alert-info">
Créer un DataFrame à partir d'un RDD est théorique et permet de comprendre la structure sous-jacente. En général, la création d'un DataFrame se fait en une seule ligne en important un fichier CSV.
</div>

### 2.2. Créer un DataFrame à partir d'un fichier CSV

La fonction <code>read.csv</code> de __SparkSession__ permet de créer un DataFrame à partir d'un fichier CSV. Cette fonction permet de spécifier s'il existe une première ligne contenant les noms de variables en utilisant l'option <code>header</code> :
```python
header = True   # Cet argument signifie que le DataFrame contient une première 
                # ligne contenant les noms des variables
```

* __(g)__ Importer la base de données 2008.csv avec le header dans un DataFrame __raw_df__.

In [5]:
# Lecture du fichier '2008.csv'
raw_df = spark.read.csv('data/2008.csv', header=True)
raw_df.show(5)

+-----+----+-----+-----+-------------+---------+-------+------+----+--------+--------+----------------+--------+------------+------------+--------+-------------+-----------------+
|annee|mois|jours|heure|uniqueCarrier|flightNum|tailNum|origin|dest|distance|canceled|cancellationCode|diverted|carrierDelay|weatherDelay|nasDelay|securityDelay|lateAircraftDelay|
+-----+----+-----+-----+-------------+---------+-------+------+----+--------+--------+----------------+--------+------------+------------+--------+-------------+-----------------+
| 2008|   1|    1| 2057|           AS|      324| N306AS|   SEA| SJC|     697|       0|            null|       0|          NA|          NA|      NA|           NA|               NA|
| 2008|   1|    1|  703|           AS|      572| N302AS|   SEA| PSP|     987|       0|            null|       0|          NA|          NA|      NA|           NA|               NA|
| 2008|   1|    1| 2011|           AS|      511| N564AS|   SAN| SEA|    1050|       0|            nu

La méthode <code>printSchema</code> permet d'obtenir le schéma des variables (noms des colonnes et types inférés), afin de vérifier si les données ont été bien enregistrées.

* __(h)__ Afficher le schéma du DataFrame __raw_df__.

In [6]:
# Affichage du schéma des variables
raw_df.printSchema()

root
 |-- annee: string (nullable = true)
 |-- mois: string (nullable = true)
 |-- jours: string (nullable = true)
 |-- heure: string (nullable = true)
 |-- uniqueCarrier: string (nullable = true)
 |-- flightNum: string (nullable = true)
 |-- tailNum: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- canceled: string (nullable = true)
 |-- cancellationCode: string (nullable = true)
 |-- diverted: string (nullable = true)
 |-- carrierDelay: string (nullable = true)
 |-- weatherDelay: string (nullable = true)
 |-- nasDelay: string (nullable = true)
 |-- securityDelay: string (nullable = true)
 |-- lateAircraftDelay: string (nullable = true)



<div class="alert alert-warning">
 Spark SQL n'infère pas correctement le type des variables : toutes sont considérées de type string ce qui gêne une partie des calculs, notamment en termes d'exploration de données. Afin de faire des calculs ou d'explorer les données, il est nécessaire de changer le type de certaines colonnes.
</div>

## 3. Explorer et manipuler un DataFrame

Maintenant que les données sont dans un <code>DataFrame</code>, il est possible d'effectuer de nombreuses transformations similaires au langage SQL. Pour sélectionner les variables, il faut utiliser la méthode <code>select</code>.

__Exemple : Sélection des variables name et age__
```python
new_df = df.select('name','age')
```

* __(a)__ Créer <code>flights1</code> un <code>DataFrame</code> contenant uniquement les variables : <code>'annee', 'mois', 'jours', 'flightNum', 'origin', 'dest', 'distance', 'canceled', 'cancellationCode', 'carrierDelay'</code>.
* __(b)__ Afficher les 20 premières lignes de <code>flights1</code>.

In [7]:
# Création d'un data frame ne contenant que les variables explicatives
flights1 = raw_df.select('annee', 'mois', 'jours', 'flightNum', 'origin', 'dest', 'distance', 'canceled', 'cancellationCode', 'carrierDelay')

# Affichage de 20 premières lignes
flights1.show() # 'show' affiche 20 lignes par défaut

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|    1|      324|   SEA| SJC|     697|       0|            null|          NA|
| 2008|   1|    1|      572|   SEA| PSP|     987|       0|            null|          NA|
| 2008|   1|    1|      511|   SAN| SEA|    1050|       0|            null|           0|
| 2008|   1|    1|      376|   SEA| GEG|     224|       0|            null|          NA|
| 2008|   1|    1|      729|   TUS| SEA|    1216|       0|            null|          NA|
| 2008|   1|    1|      283|   LAX| SEA|     954|       0|            null|          NA|
| 2008|   1|    1|      211|   LAX| SEA|     954|       0|            null|          NA|
| 2008|   1|    1|      100|   ANC| PDX|    1542|       0|            null|           0|
| 2008|   1|    1|   

<div class="alert alert-success">
L'attribut columns permet d'obtenir une liste des variables. Par exemple : print(raw_df.columns) renvoie une liste complète des variables de df.
</div>

Spark SQL offre également la structure Columns. Cette structure s'obtient en tapant le nom d'une colonne comme attribut du DataFrame séparée par un point '.'.

__Exemple : sélection des colonnes name et age__

```python
df.name
df.age
```

Cette méthode de sélection de variables, certes plus lourde mais plus riche, offre une deuxième façon de sélectionner les variables.

__Exemple : utilisation de select pour sélectionner des colonnes__

```python
new_df = df.select( df.name, df.age )
```

Cette structure est intéressante, en particulier grâce à la méthode cast des Columns. Cette méthode permet de spécifier un type particulier d'une colonne. Lors de l'affichage du schéma d'un DataFrame, toutes les variables sont considérées comme de type string, ce qui ne correspond pas toujours au type souhaité en pratique.

__Exemple : cast pour changer le type__

```python
new_df = df.select(df.name.cast("string"),
                   df.age.cast("int"))
```

* __(c)__ Créer le DataFrame flights avec les mêmes variables que fligths1 en spécifiant pour chaque colonne le type adapté.
* __(d)__ Afficher les 20 premières lignes de flights.

In [8]:
# Création d'un DataFrame en spécifiant le type des colonnes
flights = raw_df.select(raw_df.annee.cast("int"),
                        raw_df.mois.cast("int"),
                        raw_df.jours.cast("int"),
                        raw_df.flightNum.cast("int"),
                        raw_df.origin.cast("string"),
                        raw_df.dest.cast("string"),
                        raw_df.distance.cast("int"),
                        raw_df.canceled.cast("boolean"),
                        raw_df.cancellationCode.cast("string"),
                        raw_df.carrierDelay.cast("int"))

# Affichage de 20 premières lignes
flights.show()

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|    1|      324|   SEA| SJC|     697|   false|            null|        null|
| 2008|   1|    1|      572|   SEA| PSP|     987|   false|            null|        null|
| 2008|   1|    1|      511|   SAN| SEA|    1050|   false|            null|           0|
| 2008|   1|    1|      376|   SEA| GEG|     224|   false|            null|        null|
| 2008|   1|    1|      729|   TUS| SEA|    1216|   false|            null|        null|
| 2008|   1|    1|      283|   LAX| SEA|     954|   false|            null|        null|
| 2008|   1|    1|      211|   LAX| SEA|     954|   false|            null|        null|
| 2008|   1|    1|      100|   ANC| PDX|    1542|   false|            null|           0|
| 2008|   1|    1|   

<div class="alert alert-warning">
L'attribut columns permet d'obtenir une liste des variables. Par exemple : print(raw_df.columns) renvoie une liste complète des variables de df.
</div>

Comme pour les RDD, il est possible de compter le nombre de lignes grâce à la méthode count. La méthode distinct, appliquée à une seule variable, permet de filtrer tous les doublons.

__Exemple : Compter le nombre de modalités d'une variable__

```python
df.select('age').distinct().count()
```

* __(e)__ Déterminer les numéros de vols distincts dans flights.

In [9]:
# Calcul du nombre de vols ayant des numéros de vol distincts
flights.select('flightNum').distinct().count()

382

La méthode describe permet d'obtenir un résumé riche en informations pour un DataFrame. En effet, cette méthode génère un DataFrame qui s'affiche avec la méthode show.

La méthode show possède une option truncate = n qui tronque les résultats au n-ième caractère. Cela permet notamment d'afficher correctement un tableau contenant trop de décimales.

__Exemple : Affichage du résumé avec troncature__

```python
df.describe().show(truncate = 8)
```

* __(f)__ Afficher un résumé des données du DataFrame flights.
* __(g)__ Tronquer les données de façon à ce que le DataFrame s'affiche correctement.

In [10]:
## Première méthode
# Affichage d'un résumé en utilisant l'option truncate de la méthode show
flights.describe().show(truncate = 8)

+-------+------+--------+--------+---------+------+-----+--------+----------------+------------+
|summary| annee|    mois|   jours|flightNum|origin| dest|distance|cancellationCode|carrierDelay|
+-------+------+--------+--------+---------+------+-----+--------+----------------+------------+
|  count| 16830|   16830|   16830|    16830| 16830|16830|   16830|             457|        3960|
|   mean|2008.0|1.243...|13.38...| 354.7...|  null| null|923.4...|            null|    15.07...|
| stddev|   0.0|0.429...|9.085...| 232.6...|  null| null|586.1...|            null|    38.19...|
|    min|  2008|       1|       1|        1|   ADK|  ADK|      31|               A|           0|
|    max|  2008|       2|      31|      954|   YAK|  YAK|    2846|               C|         664|
+-------+------+--------+--------+---------+------+-----+--------+----------------+------------+



In [11]:
## Deuxième méthode
# Affichage d'un résumé en utilisant la méthode toPandas
flights.describe().toPandas()

,summary,annee,mois,jours,flightNum,origin,dest,distance,cancellationCode,carrierDelay
0,count,16830,16830,16830,16830,16830,16830,16830,457,3960
1,mean,2008.0,1.2436720142602495,13.3825311942959,354.79304812834226,None,None,923.4129530600119,None,15.077272727272728
2,stddev,0.0,0.4293098121645594,9.08538688270527,232.68555546399764,None,None,586.1537061300132,None,38.19684314978276
3,min,2008,1,1,1,ADK,ADK,31,A,0
4,max,2008,2,31,954,YAK,YAK,2846,C,664


<div class="alert alert-info">
Pour ne pas avoir à passer du temps sur la troncature de données et se simplifier la tâche, il existe la méthode toPandas qui permet de transformer un DataFrame Spark de petite taille en DataFrame Pandas.
La méthode toPandas est à utiliser avec des DataFrames de petite taille tels que des résumés d'informations, sinon elle peut affecter la distribution des données.
</div>

Dans la question précédente, la variable 'canceled' n'apparaît pas dans la description. De même, la description donne peu d'informations sur les variables catégorielles.

Pour les variables qualitatives, il est plus intéressant de faire apparaître la fréquence des modalités. La méthode groupBy permet de grouper les données selon une variable puis d'y appliquer une transformation supplémentaire telle que count.

* __(h)__ Résumer la variable cancellationCode en affichant le nombre d'observations de chaque modalité.

In [12]:
# Affichage du résumé de la variable catégorielle 'cancellationCode'
flights.groupBy('cancellationCode').count().show()

+----------------+-----+
|cancellationCode|count|
+----------------+-----+
|            null|16373|
|               B|  132|
|               C|   26|
|               A|  299|
+----------------+-----+



* __(i)__ Résumer ensemble les variables 'cancellationCode' et 'canceled' pour vérifier la cohérence entre ces deux variables.

In [13]:
# Affichage du résumé de la variable catégorielle 'cancellationCode' et 'canceled'
flights.groupBy('cancellationCode', 'canceled').count().show()

+----------------+--------+-----+
|cancellationCode|canceled|count|
+----------------+--------+-----+
|               A|    true|  299|
|               C|    true|   26|
|               B|    true|  132|
|            null|   false|16373|
+----------------+--------+-----+



Une autre manière de sélectionner de variables est la méthode filter. Elle permet de filtrer les données par rapport à certaines conditions.

__Exemple : Filtrage avec condition__

```python
df.filter( df.age < 20 )
# renvoie un DataFrame contenant uniquement les individus de moins de 20 ans
```

* __(j)__ Afficher les premières lignes de flights ne contenant que les vols annulés pour la raison 'C'.

In [14]:
# Affichage des 20 premièrs vols annulés pour la raison "C"
flights.filter(flights.cancellationCode == 'C').show()

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|    5|      345|   SFO| PDX|     550|    true|               C|        null|
| 2008|   1|    8|      345|   SFO| PDX|     550|    true|               C|        null|
| 2008|   1|    8|      526|   SEA| SFO|     679|    true|               C|        null|
| 2008|   1|   10|      526|   SEA| SFO|     679|    true|               C|        null|
| 2008|   1|   10|      345|   SFO| PDX|     550|    true|               C|        null|
| 2008|   1|   11|      345|   SFO| PDX|     550|    true|               C|        null|
| 2008|   1|   11|      526|   SEA| SFO|     679|    true|               C|        null|
| 2008|   1|   22|      526|   SEA| SFO|     679|    true|               C|        null|
| 2008|   1|   22|   

Filtrer et grouper les données est très important dans la manipulation des données. Ces étapes permettent de comprendre une base de données et d'en tirer des conclusions :

* __(k)__ Quel est le mois où il y a le plus d'annulations de vols ?

In [15]:
# 1ère méthode
flights.filter((flights.cancellationCode == 'A') | 
               (flights.cancellationCode == 'B') |
               (flights.cancellationCode == 'B')).show(5)

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|    1|      154|   ANC| SEA|    1449|    true|               A|        null|
| 2008|   1|    1|      327|   SNA| PDX|     859|    true|               A|        null|
| 2008|   1|    1|      488|   PDX| SNA|     859|    true|               A|        null|
| 2008|   1|    1|      464|   SEA| ONT|     956|    true|               A|        null|
| 2008|   1|    1|      631|   LAS| SEA|     866|    true|               A|        null|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
only showing top 5 rows



In [16]:
# 2ème méthode
li = ['A', 'B', 'C']
flights.filter(flights.cancellationCode.isin(li)).show(5)

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|    1|      154|   ANC| SEA|    1449|    true|               A|        null|
| 2008|   1|    1|      327|   SNA| PDX|     859|    true|               A|        null|
| 2008|   1|    1|      488|   PDX| SNA|     859|    true|               A|        null|
| 2008|   1|    1|      464|   SEA| ONT|     956|    true|               A|        null|
| 2008|   1|    1|      631|   LAS| SEA|     866|    true|               A|        null|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
only showing top 5 rows



In [18]:
# 3ème méthode => la meilleure
flights.filter(flights.canceled == True).groupBy('mois').count().show()

+----+-----+
|mois|count|
+----+-----+
|   1|  355|
|   2|  102|
+----+-----+



## 4. Création et aggrégation de variables

La méthode withColumn permet de créer une nouvelle colonne :

__Exemple : Création des colonnes ageInMonth et isMinor__

```python
df.withColumn( 'ageInMonth', df.age * 12) # Variable entière contenant l'âge d'une personne en mois
df.withColumn( 'isMinor', df.age < 18 )   # Variable booléenne indiquant si une personne est mineure ou pas
```

* __(a)__ Créer une nouvelle variable booléenne 'isLongFlight' qui vaut True si le vol parcourt une distance supérieure à 1000 miles, ou False sinon.
* __(b)__ Afficher les 10 premières lignes de flights.

In [19]:
# Création d'une nouvelle variable 'isLongFlight' et affichage des 10 premières lignes
flights.withColumn('isLongFlight', flights.distance > 1000 ).show(10)

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|isLongFlight|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+------------+
| 2008|   1|    1|      324|   SEA| SJC|     697|   false|            null|        null|       false|
| 2008|   1|    1|      572|   SEA| PSP|     987|   false|            null|        null|       false|
| 2008|   1|    1|      511|   SAN| SEA|    1050|   false|            null|           0|        true|
| 2008|   1|    1|      376|   SEA| GEG|     224|   false|            null|        null|       false|
| 2008|   1|    1|      729|   TUS| SEA|    1216|   false|            null|        null|        true|
| 2008|   1|    1|      283|   LAX| SEA|     954|   false|            null|        null|       false|
| 2008|   1|    1|      211|   LAX| SEA|     954|   false|            null|       

<div class="alert alert-warning">
L'enregistrement de la nouvelle colonne ne s'effectue nulle part. A cause du caractère immuable, aucune modification ne se fait par remplacement (in place). Pour enregistrer une nouvelle variable, il faut créer un nouvel objet ou de la créer dès la création du DataFrame.
</div>

## 5. Gestion des valeurs manquantes

Les valeurs manquantes apparaissent comme null dans la base de données. Il existe des fonctions telles que dropna ou fillna, comme présenté dans le module Pandas, ayant la syntaxe suivante :

```python
df.fillna(newValue, 'columnName')
```

__Exemple : Remplacement des noms non renseignés par 'unknown' et des âges non renseignés par 23__

```python
df.fillna('unknown', 'name') # donne un nom 'unknown' aux inconnus
df.fillna(23, 'age')         # assigne l’âge des personnes dont l’âge est inconnu à 23
```

* __(a)__ Remplacer les valeurs manquantes de 'carrierDelay' par des 0.
* __(b)__ Afficher les 6 premières lignes de flights.

In [20]:
# Remplacement des valeurs manquantes par des 0 et affichage des 6 premières lignes
flights = flights.fillna(0, 'carrierDelay')
flights.show(6)

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|    1|      324|   SEA| SJC|     697|   false|            null|           0|
| 2008|   1|    1|      572|   SEA| PSP|     987|   false|            null|           0|
| 2008|   1|    1|      511|   SAN| SEA|    1050|   false|            null|           0|
| 2008|   1|    1|      376|   SEA| GEG|     224|   false|            null|           0|
| 2008|   1|    1|      729|   TUS| SEA|    1216|   false|            null|           0|
| 2008|   1|    1|      283|   LAX| SEA|     954|   false|            null|           0|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
only showing top 6 rows



Remplacer n'importe quelle valeur est également possible en utilisant la méthode replace avec l'une des syntaxes suivantes :

```python
df.replace(oldValue, newValue)
# remplace sur l'ensemble de la base

df.replace(oldValue, newValue, 'columnName')
# remplace uniquement sur les colonnes spécifiées

df.replace([oldValue1, oldValue2], [newValue1, newValue2], 'columnName')
# si plusieurs valeurs à remplacer
```
    
* __(c)__ Remplacer les codes d'annulation 'A','B','C' respectivement par '1', '2' et '3'.

In [21]:
# Remplacement des codes d'annulation
flights = flights.replace(['A','B','C'],['1','2','3'],'cancellationCode')
flights.show()

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|    1|      324|   SEA| SJC|     697|   false|            null|           0|
| 2008|   1|    1|      572|   SEA| PSP|     987|   false|            null|           0|
| 2008|   1|    1|      511|   SAN| SEA|    1050|   false|            null|           0|
| 2008|   1|    1|      376|   SEA| GEG|     224|   false|            null|           0|
| 2008|   1|    1|      729|   TUS| SEA|    1216|   false|            null|           0|
| 2008|   1|    1|      283|   LAX| SEA|     954|   false|            null|           0|
| 2008|   1|    1|      211|   LAX| SEA|     954|   false|            null|           0|
| 2008|   1|    1|      100|   ANC| PDX|    1542|   false|            null|           0|
| 2008|   1|    1|   

La fonction orderBy fonctionne comme pour le langage SQL. Appliquée à un DataFrame, elle permet de l'ordonner selon les valeurs d'une de ses variables.

__Exemple : Ordonner df par âge croissant puis par âge décroissant__

```python
df.orderBy(df.age)
# ordonne par la variable 'age'

df.orderBy(df.age.desc())
# ordonne de façon décroissante
```

* __(d)__ Afficher les premières lignes de la base de données ordonnées de façon décroissante par le numéro de vol.

In [22]:
# Ordonner le data frame par numéro de vol décroissant
flights = flights.orderBy(flights.flightNum.desc())
flights.show()

+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
|annee|mois|jours|flightNum|origin|dest|distance|canceled|cancellationCode|carrierDelay|
+-----+----+-----+---------+------+----+--------+--------+----------------+------------+
| 2008|   1|   18|      954|   GEG| SEA|     224|    true|               1|           0|
| 2008|   2|    7|      954|   GEG| SEA|     224|   false|            null|           0|
| 2008|   1|   19|      954|   GEG| SEA|     224|   false|            null|           0|
| 2008|   1|   27|      954|   GEG| SEA|     224|   false|            null|           0|
| 2008|   2|    6|      954|   GEG| SEA|     224|   false|            null|           0|
| 2008|   1|   28|      954|   GEG| SEA|     224|   false|            null|           0|
| 2008|   1|   15|      954|   GEG| SEA|     224|   false|            null|           0|
| 2008|   1|   29|      954|   GEG| SEA|     224|   false|            null|           0|
| 2008|   1|   17|   

## 6. Requêtes SQL

Spark SQL permet également d'utiliser le langage SQL. Il est possible de faire fonctionner PySpark à l'aide de la méthode sql.

<div class="alert alert-warning">
Utiliser la méthode sql est plus lent que les méthodes ci-dessus et peut entraîner un temps de calcul significativement plus long sur des bases de données massives. La documentation de pyspark-sql contient tous les concepts SQL non évoqués jusqu'ici.
</div>

La première étape consiste à créer une vue SQL (SQL view), référencée dans le code SQL grâce à la méthode createOrReplaceTempView.

__Exemple : Création d'une vue et utilisation de la méthode SQL pour envoyer une requête__

```python
df.createOrReplaceTempView("people")
sqlDF = spark.sql("SELECT * FROM people")
```

* __(a)__ Créer une vue SQL de flights que l'on appellera "flightsView".
* __(b)__ Créer un DataFrame appelé sqlDF contenant uniquement la variable carrierDelay grâce à une requête SQL.
* __(c)__ Afficher les premières lignes de sqlDF.

In [23]:
# Création d'une vue SQL
flights.createOrReplaceTempView("flightsView")

# Création d'un DataFrame ne contenant que la variable "flightsView"
sqlDF = spark.sql("SELECT carrierDelay FROM flightsView")

# Affichage des 10 premières lignes
sqlDF.show(10)

+------------+
|carrierDelay|
+------------+
|           0|
|           0|
|           0|
|           0|
|           0|
|           0|
|          89|
|           0|
|          36|
|           0|
+------------+
only showing top 10 rows



## 7. Sample & astuces d'affichage

L'inconvénient de la méthode show est qu'elle a un mauvais rendu lorsqu'une base de données contient un grand nombre de variables. Il est possible d'utiliser la méthode toPandas pour corriger ce problème.

Cette méthode ne fonctionne que sur une base de données de petite taille.

Pour cela, la méthode sample renvoie un extrait des données en prenant essentiellement 3 arguments :
* withReplacement : un booléen qui vaut False si l'on veut un tirage sans remise et True si l'on veut un tirage avec.
* fraction : la fraction des données à conserver.
* seed : un entier quelconque qui permet de reproduire les résultats: pour un même seed, une fonction, bien qu'aléatoire, donnera toujours les mêmes résultats.

__Exemple : Un extrait du DataFrame contenant 1% des données aléatoirement choisies__

```python
df.sample(False, .01, seed = 1234)
```

* __(a)__ Afficher, de façon élégante, une dizaine de lignes de la base de données.

In [24]:
# Affichage d'un dizaine de lignes de la base de données
flights.sample(False, .0001, seed = 222).toPandas()

,annee,mois,jours,flightNum,origin,dest,distance,canceled,cancellationCode,carrierDelay
0,2008,2,10,153,ANC,OTZ,549,False,None,0
1,2008,2,4,130,ANC,ORD,2846,False,None,0
2,2008,1,15,119,SEA,ANC,1449,False,None,0


<div class="alert alert-success">
La méthode sample n'est pas simplement utile pour l'affichage, elle est également importante pour tester différentes méthodes lorsque la puissance de calcul disponible est trop faible.
</div>

* __(b)__ Fermer la session spark en utilisant la méthode stop

In [25]:
# Fermeture de la session Spark
spark.stop()

Vous avez maintenant toutes les clés nécessaires en main pour effectuer de l'exploration de données avec une performance optimale grâce à PySpark. Vous êtes donc prêt à aborder Spark ML, le module permettant de programmer des algorithmes de Machine Learning utilisant la structure DataFrame.